In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110590

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,132210.7596
2018-02-28,110256.5018
2018-03-31,126286.0143
2018-04-30,161128.0046
2018-05-31,135308.2614
2018-06-30,156514.0861
2018-07-31,177480.1947
2018-08-31,168527.4927
2018-09-30,210319.3945


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    132210.7596
 2018-02-28    110256.5018
 2018-03-31    126286.0143
 2018-04-30    161128.0046
 2018-05-31    135308.2614
 2018-06-30    156514.0861
 2018-07-31    177480.1947
 2018-08-31    168527.4927
 2018-09-30    210319.3945
 2018-10-31    150495.3081
 2018-11-30    160436.0003
 2018-12-31    164054.7713
 2019-01-31    175572.3327
 2019-02-28    163244.5104
 2019-03-31    214085.2191
 2019-04-30    161316.6200
 2019-05-31    186379.0355
 2019-06-30    177703.1091
 2019-07-31    205842.1958
 2019-08-31    234873.0025
 2019-09-30    218430.8014
 2019-10-31    204594.4159
 2019-11-30    234758.7463
 2019-12-31    293002.4081
 2020-01-31    203853.1370
 2020-02-29    172804.4690
 2020-03-31    183510.1760
 2020-04-30    150007.5265
 2020-05-31    169099.2164
 2020-06-30    212064.2967
 2020-07-31    214155.7827
 2020-08-31    196570.7829
 2020-09-30    140400.6891
 2020-10-31    164257.3891
 2020-11-30    224856.4911
 2020-12-31    184189.4760

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.569161
p-value : 0.006379
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1002.860, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=994.621, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=994.612, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=996.266, Time=0.04 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=998.469, Time=0.06 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1000.588, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=990.838, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=993.010, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=994.928, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=996.788, Time=0.09 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=998.893, Time=0.09 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=993.044, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=994.788, Time=0.15 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=996.720, Time=0.17 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=998.397, Time=0.23 sec
 ARIMA(3,0,0)(0,0,0)[1]

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -492.406
Method:                       css-mle   S.D. of innovations          29760.816
Date:                Sat, 11 Sep 2021   AIC                            990.811
Time:                        04:07:28   BIC                            996.024
Sample:                             0   HQIC                           992.722
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.767e+05   9680.603     18.251      0.000    1.58e+05    1.96e+05
ar.L1.y        0.5373      0.130      4.133      0.000       0.282       0.792
                                    Roots           

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([171807.52958537, 174064.12829565]), array([29760.81591905, 33784.04319454]), array([[113477.40223351, 230137.65693724],
       [107848.6203822 , 240279.63620909]]))
